In [1]:
from typing import Dict, Any

In [2]:
from dotenv import load_dotenv
import os
from elasticsearch import helpers  # For bulk Data Uploading
from elasticsearch import Elasticsearch  # Base function for interacting with Elasticsearch

load_dotenv()
client = Elasticsearch("http://localhost:9200/", api_key=os.getenv('apikey'))

#test client
print(client.info())

{'name': 'avior.ster.kuleuven.be', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'DimT7XnKRRiTi8I5zmSqxQ', 'version': {'number': '8.15.3', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': 'f97532e680b555c3a05e73a74c28afb666923018', 'build_date': '2024-10-09T22:08:00.328917561Z', 'build_snapshot': False, 'lucene_version': '9.11.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [3]:
'''
"settings" is technically not needed if we are working on a simple local host, but can be changed to optimise search performance on a database that is hosted on a cluster and searched by multiple users.
"mappings" is required if you wish to explicitly map fields to specific values
'''

index_definition = {
    "settings": {  
        "number_of_shards": 1,
    },
    "mappings": {
        "properties": {
            "code hash": {"type": "keyword"},
            "path to folder": {"type": "keyword"},
            "simulation time": {
                "type": "float",
                "meta": {
                    "unit": "s",
                }
            },
            "particle mass": {
                "type": "float",
                "meta": {
                    "unit": "Msun",
                }
            },
            "tmax": {"type": "float"},
            "dtmax": {"type": "float"},
            "nfulldump": {"type": "integer"},
            "eccentricity": {"type": "float"},
            "semi-major axis": {
                "type": "float",
                "meta": {
                    "unit": "s",
                }
            },
            "wind mass rate": {
                "type": "float",
                "meta": {
                    "unit": "Msun/yr",
                }
            },
            "init wind velocity": {
                "type": "float",
                "meta": {
                    "unit": "km/s",
                }
            },
            "term wind velocity": {
                "type": "float",
                "meta": {
                    "unit": "km/s",
                }
            },
            "wind inject radius": {
                "type": "float",
                "meta": {
                    "unit": "au",
                }
            },
            "wind temperature": {
                "type": "float",
                "meta": {
                    "unit": "K",
                }
            },
            "iwind_res": {"type": "integer"},
            "iwind_sh_spacing": {"type": "float"},
            "primary mass": {
                "type": "float",
                "meta": {
                    "unit": "Msun",
                }
            },
            "primary Racc": {
                "type": "float",
                "meta": {
                    "unit": "Rsun",
                }
            },
            "primary Reff": {
                "type": "float",
                "meta": {
                    "unit": "Rsun",
                }
            },
            "primary Teff": {
                "type": "float",
                "meta": {
                    "unit": "K",
                }
            },
            "secondary mass": {
                "type": "float",
                "meta": {
                    "unit": "Msun",
                }
            },
            "secondary Racc": {
                "type": "float",
                "meta": {
                    "unit": "Rsun",
                }
            },
            "secondary Reff": {
                "type": "float",
                "meta": {
                    "unit": "Rsun",
                }
            },
            "secondary Teff": {
                "type": "float",
                "meta": {
                    "unit": "K",
                }
            },
            "wind polytropic index": {"type": "float"},
            "mean molecular weight": {"type": "float"},
            "icooling": {"type": "integer"},
            "icool_method": {"type": "integer"},
            "excitation_HI": {"type": "integer"},
            "equation of state": {"type": "integer"},
            "outer boundary": {
                "type": "float",
                "meta": {
                    "unit": "au",
                }
            },
            "idust_opacity": {"type": "integer"},
            "isink_radiation": {"type": "integer"},
            "iget_tdust": {"type": "integer"},
            "f_acc": {"type": "float"},
            "Tfloor": {
                "type": "float",
                "meta": {
                    "unit": "K",
                }
            },
        }
    }
}


In [6]:
client.indices.create(index="test_index", body=index_definition)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'test_index'})

# Kibana

Once the index is created in elasticsearch, in order to view it in Kibana, go to **Management** and on left menu bar, scroll down to the **Kibana** subsection, and click **Data Views**. From here, on the top right, click **Create data view** in order to integrate the new index into the Kibana interface. This will allow you to view how kibana interprets the index you have created.

### Load information from .setup and .in file




In [7]:
def LoadSetupData(dir: str, prefix: str,index_definition) -> Dict[str, Any]:
    """ Load the prefix.in and prefix.setup files to get general information about the phantom model

    Args:
        dir (str): directory of the simulation
        prefix (str): prefix used for the files

    Returns:
        Dict[str, Any]: a dictionary containing the info from the setup files
        (!! check units, they are not all in SI or cgs)
    """

    # load the prefix.in & prefix.setup file
    setup = {}
    try:
        with open(os.path.join(dir,'%s.setup'%prefix), 'r') as f:
            lines = f.readlines()
            for string in lines:
                line = string.split()
                if len(line) != 0:
                    if line[0] != '#':
                        stringName = line[0]

                        # Names that have to be changed
                        # Floats
                        if stringName == 'primary_mass': stringName = 'primary mass'
                        elif stringName == 'primary_racc': stringName = 'primary Racc'
                        elif stringName == 'primary_Reff': stringName = 'primary Reff'
                        elif stringName == 'primary_Teff': stringName = 'primary Teff'
                        elif stringName == 'secondary_mass': stringName = 'secondary mass'
                        elif stringName == 'secondary_racc': stringName = 'secondary Racc'
                        elif stringName == 'secondary_Reff': stringName = 'secondary Reff'
                        elif stringName == 'secondary_Teff': stringName = 'secondary Teff'
                        elif stringName == 'semi_major_axis': stringName = 'semi-major axis'
                        elif stringName == 'wind_gamma' or stringName == "temp_exponent": stringName = 'wind polytropic index'
                        # # For triples
                        # elif stringName == 'secondary_mass': stringName = 'massComp_ini'
                        # elif stringName == 'binary2_a' : stringName = 'sma_in_ini'
                        # elif stringName == 'binary2_e' : stringName = 'ecc_in'
                        # elif stringName == 'secondary_racc': stringName = 'rAccrComp'
                        # elif stringName == 'accr2b' : stringName = 'rAccrComp_in'
                        # elif stringName == 'racc2b' : stringName = 'rAccrComp_in'
                        # if name is in our index dictionary, and it has type float, save it as a float
                        if (stringName in index_definition["mappings"]["properties"]) and (index_definition["mappings"]["properties"][stringName]['type']=='float'):
                            setup[stringName] = float(line[2])
                        elif (stringName in index_definition["mappings"]["properties"]) and (index_definition["mappings"]["properties"][stringName]['type']=='integer'):
                            setup[stringName] = int(line[2])
                        elif (stringName in index_definition["mappings"]["properties"]) and (index_definition["mappings"]["properties"][stringName]['type']=='string'):
                            setup[stringName] = str(line[2])

                        # # For triples
                        # if stringName == 'q2':
                        #     stringName = 'massComp_in_ini'
                        #     if stringName in index_definition:
                        #         setup[stringName] = float(line[2])*setup['primary mass']

                        # Boolean
                        if stringName == 'icompanion_star':
                            stringName = 'single_star'
                            if int(line[2]) == 0: setup[stringName] = True
                            else: setup[stringName] = False
                            stringName = 'triple_star'
                            if int(line[2]) == 2: setup[stringName] = True
                            else: setup[stringName] = False

    except FileNotFoundError:
        print('')
        print(" ERROR: No %s.setup file found!"%prefix)
        print('')
        exit()

    try:
        with open(os.path.join(dir,'%s.in'%prefix), 'r') as f:
            lines = f.readlines()
            for string in lines:
                line = string.split()
                if len(line) != 0:
                    if line[0] != '#':
                        stringName = line[0]

                        # Strings
                        # if stringName == 'logfile': setup[stringName] = str(line[2])
                        # elif stringName == 'dumpfile': setup[stringName] = str(line[2])
                        # elif stringName == 'twallmax': setup[stringName] = str(line[2])
                        # elif stringName == 'dtwallmax': setup[stringName] = str(line[2])

                        # Floats
                        # else:

                        if stringName == 'wind_mass_rate': stringName = 'wind mass rate'
                        elif stringName == 'wind_velocity': stringName = 'init wind velocity'
                        elif stringName == 'wind_inject_radius': stringName = 'wind inject radius'
                        elif stringName == 'wind_temperature': stringName = 'wind temperature'
                        elif stringName == 'mu': stringName = 'mean molecular weight'
                        elif stringName == 'ieos': stringName = 'equation of state'
                        elif stringName == 'outer_boundary': stringName = 'outer boundary'
                        # not sure what this is
                        if line[2]=='F': line[2] = 0
                        
                        if (stringName in index_definition["mappings"]["properties"]) and (index_definition["mappings"]["properties"][stringName]['type']=='float'):
                            setup[stringName] = float(line[2])
                        elif (stringName in index_definition["mappings"]["properties"]) and (index_definition["mappings"]["properties"][stringName]['type']=='integer'):
                            setup[stringName] = int(line[2])
                        elif (stringName in index_definition["mappings"]["properties"]) and (index_definition["mappings"]["properties"][stringName]['type']=='string'):
                            setup[stringName] = str(line[2])
    except FileNotFoundError:
        print('')
        print(" ERROR: No %s.in file found!"%prefix)
        print('')
        exit()

    return setup


In [6]:
#To add 1 model
Dir = "/lhome/jolienm/Documents/TierModels/finalModelsHIcooling/v10e00"
prefix = "wind"
testData = LoadSetupData(Dir,prefix,index_definition)
client.index(index='test_index', body=testData)
del testData

In [9]:
#To add multiple models
base_command = {
    "_index": 'test_index',
    "_op_type": "index"
}
operations = []
models = ['v10e00','v20e00','v05e00']
prefix = "wind"
for model in models:
    Dir = '/lhome/jolienm/Documents/TierModels/finalModelsHIcooling/'+str(model)
    modelData = LoadSetupData(Dir,prefix,index_definition)
    operations.append((base_command | {"_source": modelData}))
    del modelData
print(operations)
helpers.bulk(client, operations, refresh=True)
del operations

[{'_index': 'test_index', '_op_type': 'index', '_source': {'primary mass': 1.5, 'primary Racc': 1.22, 'primary Teff': 3000.0, 'primary Reff': 1.26675855, 'single_star': False, 'triple_star': False, 'secondary mass': 1.0, 'secondary Racc': 0.04, 'secondary Teff': 0.0, 'secondary Reff': 0.0, 'semi-major axis': 6.0, 'eccentricity': 0.0, 'wind polytropic index': 1.2, 'tmax': 600.0, 'dtmax': 1.0, 'nfulldump': 1, 'equation of state': 2, 'mean molecular weight': 2.38, 'icooling': 1, 'icool_method': 0, 'excitation_HI': 1, 'Tfloor': 10.0, 'f_acc': 0.8, 'init wind velocity': 10.0, 'wind inject radius': 1.3, 'wind mass rate': 1e-07, 'wind temperature': 3000.0, 'outer boundary': 250.0, 'idust_opacity': 0, 'isink_radiation': 1}}, {'_index': 'test_index', '_op_type': 'index', '_source': {'primary mass': 1.5, 'primary Racc': 1.22, 'primary Teff': 3000.0, 'primary Reff': 1.26675855, 'single_star': False, 'triple_star': False, 'secondary mass': 1.0, 'secondary Racc': 0.04, 'secondary Teff': 0.0, 'secon

### Now look into Dashboards: Analytics - Dashboards to visualise data
Documentation: https://www.elastic.co/guide/en/kibana/current/create-a-dashboard-of-panels-with-web-server-data.html

